In [ ]:
import math
import os
import random
import tempfile
import logging
import traceback
import datetime
import xml.etree.ElementTree as ET
import json
import glob
import io
import re
import sys
import xml.etree.ElementTree as ET
import pandas as pd
from collections import OrderedDict
from functools import partial
from typing import Any, DefaultDict, Dict, List, Optional, Tuple
import xml.etree.ElementTree as ET
from datasets import concatenate_datasets, load_dataset, Dataset, DatasetDict, Features, Value, Sequence, Array3D, ClassLabel
from transformers import BertTokenizerFast
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.utils.data as data
import torchvision.transforms.functional as FT
import torchvision.transforms as tvT
import torchmetrics
from einops import rearrange, reduce
import gradio as gr
from torchsummary import summary


In [ ]:
import datasets

# Replace this with the actual link to your Hugging Face dataset
dataset_name = "mrtoy/mobile-ui-design"  

# Load the dataset
dataset = datasets.load_dataset(dataset_name)

# Explore the dataset structure
print(dataset.keys())  # View the available splits ('train', 'test', etc.)

# Example from the 'train' split
example = dataset['train'][0]
print(example)  # Inspect the contents of a single example 
print(type(example))  # Check the data type 


In [ ]:
import os
import json
import urllib.request
import uuid
from PIL import Image
from tqdm import tqdm
import datasets
import torch
from transformers import BertTokenizer, BertForMaskedLM, AdamW, CLIPProcessor, CLIPModel
from torch.utils.data import DataLoader, Dataset
import faiss
import openai

# Function to explore dataset
def explore_dataset():
    # Load the dataset from Hugging Face
    dataset_dict = load_dataset("mrtoy/mobile-ui-design")

    # Access the specific dataset within the DatasetDict object
    dataset = dataset_dict["train"]

    # Display basic information about the dataset
    print("Dataset Name:", "mrtoy/mobile-ui-design")
    print("Number of Instances:", len(dataset))

    # Display the features available in the dataset
    print("Features:")
    for feature_name, feature_metadata in dataset.features.items():
        print("\t", feature_name, "-", feature_metadata)

    # Display a sample of the dataset
    print("Sample Instance:")
    sample_instance = dataset[0]
    print(sample_instance)

explore_dataset()

# Load dataset
dataset = load_dataset('mrtoy/mobile-ui-design', split='train')
print(dataset[:5]) 

# Function to preprocess and load dataset
def load_and_preprocess_dataset():
    # Load the dataset
    dataset = load_dataset("mrtoy/mobile-ui-design")

    # Preprocess the dataset if needed
    def preprocess(example):
        return {"objects": example["objects"]} 

    processed_dataset = dataset.map(preprocess, batched=True)  
    return processed_dataset 

# Define dataset class
class MyDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        objects = item.get("objects")  
        if objects is not None:
            if isinstance(objects, list):  
                text_list = [obj.get("text", "") for obj in objects]  
                text_list = [text for text in text_list if text]  
                text = " ".join(text_list)  
            elif isinstance(objects, dict):  
                text = objects.get("text", "") 
            else:
                text = ""  
        else:
            text = ""  

        # Ensure consistent text format:  
        if not isinstance(text, str): 
            text = " ".join([str(t) for t in text if t is not None]) 

        encoded_text = self.tokenizer.encode_plus(text, padding="max_length", max_length=128, truncation=True, return_tensors="pt")
        return encoded_text

# Define training parameters
num_epochs = 3
batch_size = 32
learning_rate = 5e-5

# Load and preprocess the dataset
dataset = load_and_preprocess_dataset()

# Define BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Prepare the dataset for training
train_dataset = MyDataset(dataset["train"], tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Fine-tune BERT on the dataset
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        inputs = batch["input_ids"]

        # Potential Fix: Reshape inputs if necessary
        if inputs.ndim > 2:  
           batch_size, seq_length = inputs.shape[:2] 
           inputs = inputs.reshape(batch_size, seq_length)

        # Debugging: Inspect input shapes
        print(inputs.shape) 

model.save_pretrained("fine_tuned_bert") 

# Function to download a file
def download_file(url):
    filename = url.split("/")[-1].split("#")[0]
    filepath = os.path.join(os.getcwd(), "files", filename)
    if os.path.exists(filepath):
        print(f"{filename} already exists.")
    else:
        urllib.request.urlretrieve(url, filepath)
        print(f"{filename} downloaded successfully.")

# Function to read JSON file
def read_json(filepath):
    with open(filepath, 'r') as fp:
        data = json.load(fp)
    return data

# Function to write JSON file
def write_json(data, filepath):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

# Function to convert data to COCO format
def convert_to_coco(processed_data, save_path):
    coco_output = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    category_set = set()
    image_id = 0
    annotation_id = 0

    for item in processed_data:
        image_data = {'id': image_id, 'width': item['width'], 'height': item['height'], 'file_name': item['file_name']}
        coco_output['images'].append(image_data)

        for obj in item['objects']:
            category = obj['category']
            category_set.add(category)

            coco_output['annotations'].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": list(category_set).index(category) + 1,
                "bbox": obj['bbox'],
                "area": obj['bbox'][2] * obj['bbox'][3],
                "iscrowd": 0
            })
            annotation_id += 1

    coco_output['categories'] = [{'id': i, 'name': name} for i, name in enumerate(category_set, start=1)]

    with open(save_path, 'w') as f:
        json.dump(coco_output, f)

# Main function to process dataset
def main():
    """Load dataset and process it."""
    # Load the dataset
    dataset = datasets.load_dataset('mrtoy/mobile-ui-design', split='train')

    # Initialize lists to store the processed data
    images, objects = [], []

    # Loop through the dataset items
    for item in tqdm(dataset):
        # Get image info
        width, height = item['width'], item['height']

        # Convert category_ids to categories
        categories = [dataset.features['category_id'].names[cat_id] for cat_id in item['category_id']]

        # Convert the bboxes to a list of dictionaries
        objects_list = [{"category": cat, "bbox": bb} for cat, bb in zip(categories, item['boxes'])]

        # Append image and objects info
        images.append({'id': item['id'], 'width': width, 'height': height, 'file_name': item['file_name']})
        objects.extend(objects_list)

    # Combine the images and objects into one list
    processed_data = {'images': images, 'objects': objects}

    # Convert the processed data to COCO format and save it to a .json file
    convert_to_coco(processed_data, save_path='mobile_ui_design.json')

# Function to load image
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return image

# Setup for vector storing and retrieval
def setup_vector_storage():
    openai.api_key = "API_KEY"
    IMAGE_EMBEDDING_MODEL = "openai/clip-vit-base-patch32"

    dataset = datasets.load_dataset("mrtoy/mobile-ui-design")
    vectorstore = Chroma(
        collection_name="ui_elements", 
        embedding_function=OpenAIEmbeddings(openai_api_key=openai.api_key)
    )
    store = InMemoryStore()
    retriever = MultiVectorRetriever(vectorstore=vectorstore, docstore=store, id_key="doc_id")
    llm = OpenAI(openai_api_key=openai.api_key, temperature=0.7) 
    chain = VectorDBQA.from_llm_and_retriever(llm, retriever)

# Function to get CLIP embedding of an image
def get_image_embedding(image):
    processor = CLIPProcessor.from_pretrained(IMAGE_EMBEDDING_MODEL)
    model = CLIPModel.from_pretrained(IMAGE_EMBEDDING_MODEL)
    inputs = processor(images=[image], return_tensors="pt")  
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.pooler_output.cpu().numpy()[0]  

# Function to add a single document (UI element) to the retriever
def add_document(image, bounding_boxes, labels, doc_id=None):
    doc_id = doc_id or str(uuid.uuid4())
    image_embedding = get_image_embedding(image)
    document = Document(page_content=image_embedding, metadata={"doc_id": doc_id, "bounding_boxes": bounding_boxes, "labels": labels})
    retriever.add_document(document)
    return doc_id

# Function to find similar UI elements
def find_similar_elements(query_image, num_results=3):
    query_embedding = get_image_embedding(query_image)
    results = retriever.get_relevant_documents(query_embedding, k=num_results)
    return results

# Function to generate detailed image description using OpenAI API
def generate_image_description(user_input_text, temperature=0.7):
    prompt = (
        f"User Input: {user_input_text}\n\n"
        f"Detailed Image Description:"
    )

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=250,  
        temperature=temperature,  
    )

    return response.choices[0].text.strip()

# Example usage
user_input = "A login screen with a minimalist aesthetic, featuring a blue color scheme"
detailed_description = generate_image_description(user_input)
print(detailed_description)


In [ ]:
# Import necessary libraries
import torch
import torchvision.transforms as transforms
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import GPT2Config
from transformers import ViTFeatureExtractor, ViTForImageGeneration

# Load dataset from Hugging Face
from datasets import load_dataset

# Load the Mobile UI Design dataset
dataset = load_dataset("mrtoy/mobile-ui-design")

# Define device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and tokenize the language model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model_config = GPT2Config.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", config=model_config).to(device)

# Load and preprocess the computer vision model
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
image_model = ViTForImageGeneration.from_pretrained("google/vit-base-patch16-224-in21k").to(device)

# Define image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Function to generate mobile UI design based on input pattern
def generate_mobile_ui_design(input_pattern):
    # Tokenize input pattern
    input_ids = tokenizer.encode(input_pattern, return_tensors="pt").to(device)
    
    # Generate text based on input pattern
    output = model.generate(input_ids, max_length=100, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Generate image based on generated text
    inputs = feature_extractor(images=input_pattern, return_tensors="pt").to(device)
    image = image_model.generate(**inputs)
    
    return generated_text, image

# Example usage
input_pattern = "A chat messaging app with dark mode"
generated_text, generated_image = generate_mobile_ui_design(input_pattern)
print("Generated Text:", generated_text)
# Display or save generated_image
